In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import tensorflow as tf
#import h5py
#from scipy import signal
#from tensorflow.python.keras.models import load_model
#import cv2

import matplotlib as mpl
import subprocess
import skimage
import skimage.io as imio
import skimage.transform as trans
import glob
import re
from time import gmtime, strftime, sleep
from PIL import Image
from keras.models import load_model
from scipy import signal,misc


mpl.rcParams['figure.figsize'] = (32.0, 32.0) 

In [ ]:
saliency = load_model('HED4.h5')

In [ ]:
def patchExtraction(sid = 2):
    rIm = grabImage(sid = sid)[1]
    m, n= rIm.shape
    im = rIm/(2**14-1)
    img=Image.fromarray(np.uint8(im*255))
    img = img.resize((400,300))
    inpt = np.array(img)/255
    res = saliency.predict(inpt.reshape(1,300,400,1))
    res = res.reshape(300,400)
    w=np.ones((71,53))
    res_new=signal.convolve2d(res,w,mode='same')
    pos = np.where(res_new==np.max(res_new))
    x = np.round(pos[0]*(m/300))
    y = np.round(pos[1]*(n/400))
    patch = rIm[int(x)-256:int(x)+256, int(y)-256:int(y)+256]
    patch = (patch - patch.min()) / (patch.max() - patch.min())
    return patch